# Index method scalability comparison

This notebook compares the scalability properties of AdjacentBlockIndex with the BlockIndex and SortedNeighbourhoodIndex (both in the recordlinkage package)

In [16]:
from collections import ChainMap
from functools import lru_cache, reduce
from itertools import product
from operator import mul
import datetime, inspect, pathlib
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from recordlinkage import FullIndex, BlockIndex, SortedNeighbourhoodIndex
from adjacent_block_index import AdjacentBlockIndex

from experiment_helpers import sample_dedup_dataset
sns.set_style()
%matplotlib inline

## Testing function

In [17]:
def get_index_timings(index_details, row_counts, column_counts, distinct_entity_instance_counts, granularities, index_length_limit=5e6, index_time_limit=1, partial_results=None, verbose=False, debug=False):
    '''
    Negative granularities are neighbourhood radii
    '''
    timings = pd.DataFrame() if partial_results is None else partial_results
    def msg(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)
    def debug_msg(*args, **kwargs):
        if debug:
            print(*args, **kwargs)
    total_combinations = reduce(mul, map(len, [index_details, row_counts, column_counts, distinct_entity_instance_counts, granularities]), 1)
    combinations_processed = progress = 0
    id_cols = ['instances_per_entity', 'granularity', 'rows', 'columns', 'label']
    for instances_per_entity in sorted(distinct_entity_instance_counts):
        msg('Instances per entity: {instances_per_entity}'.format(**locals()))
        largest_dataset = {'continuous': sample_dedup_dataset(rows=max(row_counts), columns=max(column_counts), instances_per_entity=instances_per_entity)}
        for granularity in sorted(granularities, reverse=True):
            msg('{progress:.0%}\tGranularity: {granularity}'.format(**locals()))
            if granularity > 0:
                largest_dataset['discrete'] = np.floor(largest_dataset['continuous'] * granularity)
            for rows, columns in product(sorted(row_counts), sorted(column_counts)):
                msg('{progress:.0%}\t\tTable: {rows} x {columns}'.format(**locals()))
                dataset = {k: v.iloc[:rows, :columns] for k, v in largest_dataset.items()}
                length_full = rows * (rows-1) // 2
                block_count = len(dataset['discrete'].drop_duplicates()) if granularity > 0 else 1
                est_length_std = rows * rows // block_count // 2
                est_length_adj = min([length_full, est_length_std * int(3 ** columns)])
                for label, index_type, index_kwargs in index_details:
                    index_kwargs = dict(index_kwargs)
                    combinations_processed += 1
                    progress = combinations_processed / total_combinations
                    if index_type is BlockIndex:
                        est_length = est_length_std
                    elif index_type is SortedNeighbourhoodIndex:
                        sw_radius = int(rows / granularity / 2) if granularity > 0 else -granularity
                        est_length = sw_radius * rows - (sw_radius * (sw_radius+1)) // 2
                    elif index_type is AdjacentBlockIndex:
                        est_length = est_length_adj
                    elif index_type is FullIndex:
                        est_length = length_full
                    else:
                        est_length = np.nan
                    _locals = locals()
                    def omission_contitions():
                        debug_msg('\t\t\t\t{granularity} inapplicable to {label}?'.format(**locals()))
                        yield ((index_type is not SortedNeighbourhoodIndex) and (granularity < 0))
                        debug_msg('\t\t\t\t{granularity} granularity exceeds {rows} rows?'.format(**locals()))
                        yield granularity > rows
                        debug_msg('\t\t\t\t{columns} Too many columns?'.format(**locals()))
                        yield ((index_type is SortedNeighbourhoodIndex) and (columns > 1))
                        debug_msg('\t\t\t\tEstimated length {est_length} too high?'.format(**locals()))
                        yield (index_type in [FullIndex, SortedNeighbourhoodIndex]) and (est_length > index_length_limit)
                        debug_msg('\t\t\t\tmax nulls too high?'.format(**locals()))
                        yield index_kwargs.get('max_nulls', columns) > columns
                        if len(timings) > 0:
                            debug_msg('\t\t\t\tthis config already done?'.format(**locals()))
                            yield reduce((lambda a,b: a&b), (timings[c]==_locals[c] for c in id_cols)).any()
                            if index_type is FullIndex:
                                debug_msg('\t\t\t\t{rows} rows already done for {label}?'.format(**locals()))
                                yield (timings[timings['label']==label]['rows'] == rows).any()
                            if index_type is SortedNeighbourhoodIndex:
                                debug_msg('\t\t\t\t{rows} rows {granularity} granularity already done for {label}?'.format(**locals()))
                                yield ((timings[timings['label']==label]['rows'] == rows) & (timings[timings['label']==label]['granularity'] == granularity)).any()
                            #comparison_rows = (timings['label'] == label) & (timings['columns'] == columns) & (np.isnan(est_length) | (timings['est_length']<= est_length)) 
                            comparison_rows = (timings['label'] == label) & (timings['rows'] <= rows) & (timings['columns'] == columns) & (timings['granularity'] >= granularity) & (timings['instances_per_entity'] <= instances_per_entity)
                            if comparison_rows.any():
                                comparison_maxima = timings[comparison_rows][['elapsed_seconds', 'index_length']].max()
                                if index_time_limit is not None:
                                    debug_msg('\t\t\t\tsmaller index already exceeds time limit?')
                                    yield comparison_maxima['elapsed_seconds'] > index_time_limit
                                if index_length_limit is not None:
                                    debug_msg('\t\t\t\tsmaller index already exceeds length limit?')
                                    yield comparison_maxima['index_length'] > index_length_limit
                    if any(omission_contitions()):
                        debug_msg ('\t\t\t\t\tskipping {label}'.format(**locals()))
                        continue
                    msg('{progress:.0%}\t\t\t{label}'.format(**locals()))
                    stats_dict = {k: _locals[k] for k in (['est_length', 'length_full'] + id_cols)}
                    stats_dict.update(index_kwargs)
                    if index_type is SortedNeighbourhoodIndex:
                        dataset_type = 'continuous'
                        index_kwargs['window'] = 1 + 2 * sw_radius
                    else:
                        dataset_type = 'discrete'
                        stats_dict.update({k: _locals[k] for k in ['granularity', 'block_count']})
                    if 'on' in inspect.signature(index_type).parameters:
                        index_kwargs['on'] = list(dataset[dataset_type].columns)
                    indexer = index_type(**index_kwargs)
                    start_time = datetime.datetime.now()
                    index = indexer.index(dataset[dataset_type])
                    elapsed = datetime.datetime.now() - start_time
                    stats_dict.update({'index_length': len(index),
                                       'elapsed_seconds': elapsed.total_seconds(),
                                      })
                    timings = timings.append(stats_dict, ignore_index=True)
    timings['reduction_ratio'] = 1 - timings['index_length'] / timings['length_full']
    progress = 1
    msg('{progress:.0%}\t Done'.format(**locals()))
    return timings

## Execution

In [18]:
result_file = pathlib.Path('timings.pickle')
index_details = [#(label, type, additional kwargs),
                 ('Full', FullIndex, {}),
                 ('Standard Blocking',BlockIndex , {}),
                 ('Sorted Neighbourhood', SortedNeighbourhoodIndex, {}),
                 ('Adjacent Blocking - no wildcards',AdjacentBlockIndex , {'max_nulls': 0}),
                 ('Adjacent Blocking - 1 wildcard',AdjacentBlockIndex , {'max_nulls': 1}),
                 ('Adjacent Blocking - 2 wildcards',AdjacentBlockIndex , {'max_nulls': 2}),
                ]

def int_geomspace(start, stop, num=50, endpoint=True):
    return np.unique(np.logspace(np.log10(start), np.log10(stop), num=num, endpoint=endpoint, dtype=int))

neighbourhood_radii = [1, 2, 3, 4, 5, 7, 10, 15, 20, 30, 40, 50, 70, 100, 200, 300, 500, 1000]

timings = get_index_timings(index_details=index_details,
                        row_counts = int_geomspace(start=10, stop=1000000, num=16),
                        column_counts = 1 + np.arange(10),
                        distinct_entity_instance_counts = int_geomspace(start=1, stop=100, num=5),
                        granularities = (-np.array(neighbourhood_radii)).tolist() + int_geomspace(start=1, stop=1000, num=10).tolist(),
                        index_length_limit = 10e6, #8e6,
                        index_time_limit = 35, #8,
                        partial_results = pd.read_pickle(str(result_file)) if result_file.exists() else None,
                        verbose = True,
                        debug = True,
                        )
timings.to_pickle(str(result_file))

Instances per entity: 1
0%	Granularity: -1
0%		Table: 10 x 1
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 10 rows?
				1 Too many columns?
				Estimated length 9 too high?
				max nulls too high?
				this config already done?
					skipping Sorted Neighbourhood
0%		Table: 10 x 2
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 10 rows?
				2 Too many columns?
					skipping Sorted Neighbourhood
0%		Table: 10 x 3
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 10 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
0%		Table: 10 x 4
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 10 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
0%		Table: 10 x 5
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 10 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
0%		Table: 10 x 6
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularit

1%		Table: 21544 x 1
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 21544 rows?
				1 Too many columns?
				Estimated length 21543 too high?
				max nulls too high?
				this config already done?
					skipping Sorted Neighbourhood
1%		Table: 21544 x 2
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 21544 rows?
				2 Too many columns?
					skipping Sorted Neighbourhood
1%		Table: 21544 x 3
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 21544 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
1%		Table: 21544 x 4
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 21544 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
1%		Table: 21544 x 5
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exceeds 21544 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
1%		Table: 21544 x 6
				-1 inapplicable to Sorted Neighbourhood?
				-1 granularity exce

				1 Too many columns?
				Estimated length 430883 too high?
				max nulls too high?
				this config already done?
					skipping Sorted Neighbourhood
2%		Table: 215443 x 2
				-2 inapplicable to Sorted Neighbourhood?
				-2 granularity exceeds 215443 rows?
				2 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 215443 x 3
				-2 inapplicable to Sorted Neighbourhood?
				-2 granularity exceeds 215443 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 215443 x 4
				-2 inapplicable to Sorted Neighbourhood?
				-2 granularity exceeds 215443 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 215443 x 5
				-2 inapplicable to Sorted Neighbourhood?
				-2 granularity exceeds 215443 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 215443 x 6
				-2 inapplicable to Sorted Neighbourhood?
				-2 granularity exceeds 215443 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 215443 x

2%		Table: 100 x 4
				-3 inapplicable to Sorted Neighbourhood?
				-3 granularity exceeds 100 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 100 x 5
				-3 inapplicable to Sorted Neighbourhood?
				-3 granularity exceeds 100 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 100 x 6
				-3 inapplicable to Sorted Neighbourhood?
				-3 granularity exceeds 100 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 100 x 7
				-3 inapplicable to Sorted Neighbourhood?
				-3 granularity exceeds 100 rows?
				7 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 100 x 8
				-3 inapplicable to Sorted Neighbourhood?
				-3 granularity exceeds 100 rows?
				8 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 100 x 9
				-3 inapplicable to Sorted Neighbourhood?
				-3 granularity exceeds 100 rows?
				9 Too many columns?
					skipping Sorted Neighbourhood
2%		Table: 100 x 10
				-3 inapplicable 

4%		Table: 100 x 3
				-4 inapplicable to Sorted Neighbourhood?
				-4 granularity exceeds 100 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
4%		Table: 100 x 4
				-4 inapplicable to Sorted Neighbourhood?
				-4 granularity exceeds 100 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
4%		Table: 100 x 5
				-4 inapplicable to Sorted Neighbourhood?
				-4 granularity exceeds 100 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
4%		Table: 100 x 6
				-4 inapplicable to Sorted Neighbourhood?
				-4 granularity exceeds 100 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
4%		Table: 100 x 7
				-4 inapplicable to Sorted Neighbourhood?
				-4 granularity exceeds 100 rows?
				7 Too many columns?
					skipping Sorted Neighbourhood
4%		Table: 100 x 8
				-4 inapplicable to Sorted Neighbourhood?
				-4 granularity exceeds 100 rows?
				8 Too many columns?
					skipping Sorted Neighbourhood
4%		Table: 100 x 9
				-4 inapplicable t

...

				-20 granularity exceeds 10000 rows?
				2 Too many columns?
					skipping Sorted Neighbourhood
90%		Table: 10000 x 3
				-20 inapplicable to Sorted Neighbourhood?
				-20 granularity exceeds 10000 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
90%		Table: 10000 x 4
				-20 inapplicable to Sorted Neighbourhood?
				-20 granularity exceeds 10000 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
90%		Table: 10000 x 5
				-20 inapplicable to Sorted Neighbourhood?
				-20 granularity exceeds 10000 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
90%		Table: 10000 x 6
				-20 inapplicable to Sorted Neighbourhood?
				-20 granularity exceeds 10000 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
90%		Table: 10000 x 7
				-20 inapplicable to Sorted Neighbourhood?
				-20 granularity exceeds 10000 rows?
				7 Too many columns?
					skipping Sorted Neighbourhood
90%		Table: 10000 x 8
				-20 inapplicable to Sorted Neighbourhood

				-40 inapplicable to Sorted Neighbourhood?
				-40 granularity exceeds 215 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
91%		Table: 215 x 6
				-40 inapplicable to Sorted Neighbourhood?
				-40 granularity exceeds 215 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
91%		Table: 215 x 7
				-40 inapplicable to Sorted Neighbourhood?
				-40 granularity exceeds 215 rows?
				7 Too many columns?
					skipping Sorted Neighbourhood
91%		Table: 215 x 8
				-40 inapplicable to Sorted Neighbourhood?
				-40 granularity exceeds 215 rows?
				8 Too many columns?
					skipping Sorted Neighbourhood
91%		Table: 215 x 9
				-40 inapplicable to Sorted Neighbourhood?
				-40 granularity exceeds 215 rows?
				9 Too many columns?
					skipping Sorted Neighbourhood
91%		Table: 215 x 10
				-40 inapplicable to Sorted Neighbourhood?
				-40 granularity exceeds 215 rows?
				10 Too many columns?
					skipping Sorted Neighbourhood
91%		Table: 464 x 1
				-40 inapplicable

				21 rows -70 granularity already done for Sorted Neighbourhood?
					skipping Sorted Neighbourhood
93%		Table: 21 x 2
				-70 inapplicable to Sorted Neighbourhood?
				-70 granularity exceeds 21 rows?
				2 Too many columns?
					skipping Sorted Neighbourhood
93%		Table: 21 x 3
				-70 inapplicable to Sorted Neighbourhood?
				-70 granularity exceeds 21 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
93%		Table: 21 x 4
				-70 inapplicable to Sorted Neighbourhood?
				-70 granularity exceeds 21 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
93%		Table: 21 x 5
				-70 inapplicable to Sorted Neighbourhood?
				-70 granularity exceeds 21 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
93%		Table: 21 x 6
				-70 inapplicable to Sorted Neighbourhood?
				-70 granularity exceeds 21 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
93%		Table: 21 x 7
				-70 inapplicable to Sorted Neighbourhood?
				-70 granularity exceeds 

				-100 granularity exceeds 215443 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
95%		Table: 215443 x 5
				-100 inapplicable to Sorted Neighbourhood?
				-100 granularity exceeds 215443 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
95%		Table: 215443 x 6
				-100 inapplicable to Sorted Neighbourhood?
				-100 granularity exceeds 215443 rows?
				6 Too many columns?
					skipping Sorted Neighbourhood
95%		Table: 215443 x 7
				-100 inapplicable to Sorted Neighbourhood?
				-100 granularity exceeds 215443 rows?
				7 Too many columns?
					skipping Sorted Neighbourhood
95%		Table: 215443 x 8
				-100 inapplicable to Sorted Neighbourhood?
				-100 granularity exceeds 215443 rows?
				8 Too many columns?
					skipping Sorted Neighbourhood
95%		Table: 215443 x 9
				-100 inapplicable to Sorted Neighbourhood?
				-100 granularity exceeds 215443 rows?
				9 Too many columns?
					skipping Sorted Neighbourhood
95%		Table: 215443 x 10
				-100 inapplicabl

				-300 granularity exceeds 10000 rows?
				1 Too many columns?
				Estimated length 2954850 too high?
				max nulls too high?
				this config already done?
				10000 rows -300 granularity already done for Sorted Neighbourhood?
					skipping Sorted Neighbourhood
97%		Table: 10000 x 2
				-300 inapplicable to Sorted Neighbourhood?
				-300 granularity exceeds 10000 rows?
				2 Too many columns?
					skipping Sorted Neighbourhood
97%		Table: 10000 x 3
				-300 inapplicable to Sorted Neighbourhood?
				-300 granularity exceeds 10000 rows?
				3 Too many columns?
					skipping Sorted Neighbourhood
97%		Table: 10000 x 4
				-300 inapplicable to Sorted Neighbourhood?
				-300 granularity exceeds 10000 rows?
				4 Too many columns?
					skipping Sorted Neighbourhood
97%		Table: 10000 x 5
				-300 inapplicable to Sorted Neighbourhood?
				-300 granularity exceeds 10000 rows?
				5 Too many columns?
					skipping Sorted Neighbourhood
97%		Table: 10000 x 6
				-300 inapplicable to Sorted Neighb

## Results

In [19]:
timings = pd.read_pickle(str(result_file))
timings[timings['label'] == 'Sorted Neighbourhood']

,block_count,columns,elapsed_seconds,est_length,granularity,index_length,instances_per_entity,label,length_full,max_nulls,rows,reduction_ratio
1643,NaN,1.0,0.009530,0.0,100.0,0.0,1.0,Sorted Neighbourhood,4.500000e+01,NaN,10.0,1.000000
1665,NaN,1.0,0.012230,0.0,100.0,0.0,1.0,Sorted Neighbourhood,2.100000e+02,NaN,21.0,1.000000
1685,NaN,1.0,0.097952,0.0,100.0,0.0,1.0,Sorted Neighbourhood,1.035000e+03,NaN,46.0,1.000000
1705,NaN,1.0,0.095897,0.0,100.0,0.0,1.0,Sorted Neighbourhood,4.950000e+03,NaN,100.0,1.000000
1723,NaN,1.0,0.115418,214.0,100.0,210.0,1.0,Sorted Neighbourhood,2.300500e+04,NaN,215.0,0.990872
1741,NaN,1.0,0.112442,925.0,100.0,911.0,1.0,Sorted Neighbourhood,1.074160e+05,NaN,464.0,0.991519
1757,NaN,1.0,0.136357,4985.0,100.0,4920.0,1.0,Sorted Neighbourhood,4.995000e+05,NaN,1000.0,0.990150
1773,NaN,1.0,0.191155,21485.0,100.0,21215.0,1.0,Sorted Neighbourhood,2.318781e+06,NaN,2154.0,0.990851
1787,NaN,1.0,0.481618,106467.0,100.0,105202.0,1.0,Sorted Neighbourhood,1.076712e+07,NaN,4641.0,0.990229
1801,NaN,1.0,1.256803,498725.0,100.0,493175.0,1.0,Sorted Neighbourhood,4.999500e+07,NaN,10000.0,0.990136
